In [94]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\Joaquín Amat\Documents\GitHub\skforecast


SOURCES:

https://mapie.readthedocs.io/en/latest/theoretical_description_regression.html

https://medium.com/@icvandenende/leveraging-conformal-prediction-in-python-to-accelerate-the-renewable-energy-transition-09b5c855f69d

https://mindfulmodeler.substack.com/p/week-3-conformal-prediction-for-regression



In [96]:
# Data processing
# ==============================================================================
import numpy as np
import pandas as pd

# Plots
# ==============================================================================
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
pio.templates.default = "seaborn"
pio.renderers.default = 'notebook' 
poff.init_notebook_mode(connected=True)

# Modelling and Forecasting
# ==============================================================================
import skforecast
import sklearn
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_pinball_loss
from feature_engine.datetime import DatetimeFeatures
from feature_engine.creation import CyclicalFeatures
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection import TimeSeriesFold, OneStepAheadFold
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.metrics import coverage

# Warnings configuration
# ==============================================================================
import warnings
from skforecast.exceptions import OneStepAheadValidationWarning
warnings.filterwarnings('once')

print('Versión skforecast:', skforecast.__version__)
print('Versión sklearn:', sklearn.__version__)

Versión skforecast: 0.15.0
Versión sklearn: 1.5.2


In [76]:
# Load data
# ==============================================================================
data = pd.read_csv("https://raw.githubusercontent.com/skforecast/skforecast-datasets/main/data/ETTm2.csv")
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data = data.asfreq('15min')
data = data.resample(rule="1h", closed="left", label="right").mean()
data.head()

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
date,,,,,,,
2016-07-01 01:00:00,38.784501,10.88975,34.753500,8.55100,4.12575,1.26050,37.838250
2016-07-01 02:00:00,36.041249,9.44475,32.696001,7.13700,3.59025,0.62900,36.849250
2016-07-01 03:00:00,38.240000,11.41350,35.343501,9.10725,3.06000,0.31175,35.915750
2016-07-01 04:00:00,37.800250,11.45525,34.881000,9.28850,3.04400,0.60750,32.839375
2016-07-01 05:00:00,36.501750,10.49200,33.708250,8.65150,2.64400,0.00000,31.466125


In [77]:
# Calendar features
# ==============================================================================
features_to_extract = [
    'year',
    'month',
    'week',
    'day_of_week',
    'hour'
]
calendar_transformer = DatetimeFeatures(
    variables           = 'index',
    features_to_extract = features_to_extract,
    drop_original       = False,
)

# Cliclical encoding of calendar features
# ==============================================================================
features_to_encode = [
    "month",
    "week",
    "day_of_week",
    "hour",
]
max_values = {
    "month": 12,
    "week": 52,
    "day_of_week": 7,
    "hour": 24,
}
cyclical_encoder = CyclicalFeatures(
                        variables     = features_to_encode,
                        max_values    = max_values,
                        drop_original = True
                   )

exog_transformer = make_pipeline(
                        calendar_transformer,
                        cyclical_encoder
                   )
display(exog_transformer)

data = exog_transformer.fit_transform(data)
# Remove rows with NaNs created by lag features
data = data.dropna()
exog_features = data.columns.difference(['OT']).tolist()
display(data.head(3))

Pipeline(steps=[('datetimefeatures',
                 DatetimeFeatures(drop_original=False,
                                  features_to_extract=['year', 'month', 'week',
                                                       'day_of_week', 'hour'],
                                  variables='index')),
                ('cyclicalfeatures',
                 CyclicalFeatures(drop_original=True,
                                  max_values={'day_of_week': 7, 'hour': 24,
                                              'month': 12, 'week': 52},
                                  variables=['month', 'week', 'day_of_week',
                                             'hour']))])

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT,year,month_sin,month_cos,week_sin,week_cos,day_of_week_sin,day_of_week_cos,hour_sin,hour_cos
date,,,,,,,,,,,,,,,,
2016-07-01 01:00:00,38.784501,10.88975,34.753500,8.55100,4.12575,1.26050,37.83825,2016,-0.5,-0.866025,1.224647e-16,-1.0,-0.433884,-0.900969,0.258819,0.965926
2016-07-01 02:00:00,36.041249,9.44475,32.696001,7.13700,3.59025,0.62900,36.84925,2016,-0.5,-0.866025,1.224647e-16,-1.0,-0.433884,-0.900969,0.500000,0.866025
2016-07-01 03:00:00,38.240000,11.41350,35.343501,9.10725,3.06000,0.31175,35.91575,2016,-0.5,-0.866025,1.224647e-16,-1.0,-0.433884,-0.900969,0.707107,0.707107


In [78]:
selected_exog = [
    "HUFL",
    "HULL",
    "LUFL",
    "LULL",
    "MUFL",
    "MULL",
    "day_of_week_sin",
    "hour_cos",
    "hour_sin",
    "month_cos",
    "month_sin",
    "week_cos",
    "week_sin",
    "year",
]

lags = [1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 42]

In [ ]:
# Split train-validation-test
# ==============================================================================
end_train = '2017-10-01 23:59:00'
end_validation = '2018-04-03 23:59:00'
data_train = data.loc[: end_train, :]
data_val   = data.loc[end_train:end_validation, :]
data_test  = data.loc[end_validation:, :]

print(f"Dates train      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Dates validacion : {data_val.index.min()} --- {data_val.index.max()}  (n={len(data_val)})")
print(f"Dates test       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

Dates train      : 2016-07-01 01:00:00 --- 2017-10-01 23:00:00  (n=10991)
Dates validacion : 2017-10-02 00:00:00 --- 2018-04-03 23:00:00  (n=4416)
Dates test       : 2018-04-04 00:00:00 --- 2018-06-26 20:00:00  (n=2013)


## ForecasterDirect

In [80]:
# Create forecasters: one for each limit of the interval
# ==============================================================================
# The forecasters obtained for alpha=0.1 and alpha=0.9 produce a 80% confidence
# interval (90% - 10% = 80%).

# Forecaster for quantile 10%
forecaster_lower_bound = ForecasterDirect(
                            regressor = LGBMRegressor(
                                            objective    = 'quantile',
                                            metric       = 'quantile',
                                            alpha        = 0.1,
                                            random_state = 15926,
                                            verbose      = -1
                                            
                                        ),
                            lags  = lags,
                            steps = 24,
                            differentiation = 1,
                        )
# Forecaster for quantile 90%
forecaster_upper_bound = ForecasterDirect(
                            regressor = LGBMRegressor(
                                            objective    = 'quantile',
                                            metric       = 'quantile',
                                            alpha        = 0.9,
                                            random_state = 15926,
                                            verbose      = -1
                                            
                                        ),
                            lags  = lags,
                            steps = 24,
                            differentiation = 1,
                        )

In [81]:
# Loss function for each quantile (pinball_loss)
# ==============================================================================
def mean_pinball_loss_constructor(alpha: float) -> callable:
    """
    Create Pinball loss for a given quantile.

    Parameters
    ----------
    alpha: float
        Quantile.

    Returns
    -------
    mean_pinball_loss_q: callable
        Mean Pinball loss for the given quantile.
    """
    if not (0 <= alpha <= 1):
        raise ValueError("alpha must be between 0 and 1.")

    def mean_pinball_loss_q(y_true, y_pred):
        return mean_pinball_loss(y_true, y_pred, alpha=alpha)
    return mean_pinball_loss_q

mean_pinball_loss_q05 = mean_pinball_loss_constructor(alpha=0.05)
mean_pinball_loss_q10 = mean_pinball_loss_constructor(alpha=0.1)
mean_pinball_loss_q90 = mean_pinball_loss_constructor(alpha=0.9)
mean_pinball_loss_q95 = mean_pinball_loss_constructor(alpha=0.95)

In [ ]:
# Bayesian search of hyper-parameters and lags for each quantile forecaster
# ==============================================================================
warnings.simplefilter('ignore', category=OneStepAheadValidationWarning)
def search_space(trial):
    search_space  = {
        'n_estimators'  : trial.suggest_int('n_estimators', 100, 500, step=50),
        'max_depth'     : trial.suggest_categorical('max_depth', [-1, 3, 5, 7, 10]),
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.1)
    }

    return search_space

cv = OneStepAheadFold(
        initial_train_size = len(data.loc[:end_train, :]),
        differentiation    = 1,
     )

results_grid_lower_bound = bayesian_search_forecaster(
                       forecaster     = forecaster_lower_bound,
                       y              = data.loc[:end_validation, 'OT'],
                       exog           = data.loc[:end_validation, selected_exog],
                       cv             = cv,
                       metric         = mean_pinball_loss_q10,
                       search_space   = search_space,
                       n_trials       = 10,
                       random_state   = 123,
                       return_best    = True,
                       n_jobs         = 'auto',
                       verbose        = False,
                       show_progress  = True
                   )

results_grid_upper_bound = bayesian_search_forecaster(
                       forecaster    = forecaster_upper_bound,
                       y              = data.loc[:end_validation, 'OT'],
                       exog           = data.loc[:end_validation, selected_exog],
                       cv            = cv,
                       metric        = mean_pinball_loss_q90,
                       search_space  = search_space,
                       n_trials      = 10,
                       random_state  = 123,
                       return_best   = True,
                       n_jobs        = 'auto',
                       verbose       = False,
                       show_progress = True
                   )

  0%|          | 0/10 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 42] 
  Parameters: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.0982687778546154}
  One-step-ahead metric: 85.20115245882792


c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\model_selection\_search.py:729: OneStepAheadValidationWarning:

One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=OneStepAheadValidationWarning)



  0%|          | 0/10 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 42] 
  Parameters: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.0982687778546154}
  One-step-ahead metric: 80.517771780236


In [83]:
# Backtesting on test data
# ==============================================================================
cv = TimeSeriesFold(
        initial_train_size = len(data.loc[:end_validation, :]),
        steps              = 24,  # all hours of next day
        differentiation    = 1
     )
_, predictions_lower_bound = backtesting_forecaster(
                                 forecaster          = forecaster_lower_bound,
                                 y                   = data['OT'],
                                 exog                = data[selected_exog],
                                 cv                  = cv,
                                 metric              = mean_pinball_loss_q10,
                                 n_jobs              = 'auto',
                                 verbose             = False,
                                 show_progress       = True
                              )

_, predictions_upper_bound = backtesting_forecaster(
                                  forecaster          = forecaster_upper_bound,
                                  y                   = data['OT'],
                                  exog                = data[selected_exog],
                                  cv                  = cv,
                                  metric              = mean_pinball_loss_q90,
                                  n_jobs              = 'auto',
                                  verbose             = False,
                                  show_progress       = True
                              )

prediction_interval = pd.concat([predictions_lower_bound, predictions_upper_bound], axis=1)
prediction_interval.columns = ['lower_bound', 'upper_bound']
prediction_interval.head(3)

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

,lower_bound,upper_bound
2018-04-04 00:00:00,32.342958,32.882199
2018-04-04 01:00:00,31.496005,32.475559
2018-04-04 02:00:00,30.717917,32.167618


In [84]:
# Predicted interval coverage (on test data)
# ==============================================================================
empirical_coverage = coverage(
                        y_true      = data.loc[end_validation:, 'OT'].to_numpy(),
                        lower_bound = predictions_lower_bound["pred"].to_numpy(), 
                        upper_bound = predictions_upper_bound["pred"].to_numpy()
                    )
print(f"Predicted interval coverage: {round(100 * empirical_coverage, 2)} %")

# Area of the interval
# ==============================================================================
area = (prediction_interval["upper_bound"] - prediction_interval["lower_bound"]).sum()
print(f"Area of the interval: {round(area, 2)}")

Predicted interval coverage: 90.21 %
Area of the interval: 29036.6


In [85]:
prediction_interval_no_conformal = prediction_interval.copy()

# Conformal intervals

In [ ]:
# 1) Backtesting on your calibration set
# ==============================================================================
cv = TimeSeriesFold(
        initial_train_size = len(data.loc[:end_train, :]),
        steps              = 24,  # all hours of next day
        differentiation    = 1
     )
_, predictions_lower_bound = backtesting_forecaster(
                                  forecaster          = forecaster_lower_bound,
                                  y                   = data.loc[:end_validation, 'OT'],
                                  exog                = data.loc[:end_validation, selected_exog],
                                  cv                  = cv,
                                  metric              = mean_pinball_loss_q05,
                                  n_jobs              = 'auto',
                                  verbose             = False,
                                  show_progress       = True
                              )

_, predictions_upper_bound = backtesting_forecaster(
                                  forecaster          = forecaster_upper_bound,
                                  y                   = data.loc[:end_validation, 'OT'],
                                  exog                = data.loc[:end_validation, selected_exog],
                                  cv                  = cv,
                                  metric              = mean_pinball_loss_q05,
                                  n_jobs              = 'auto',
                                  verbose             = False,
                                  show_progress       = True
                              )
prediction_interval_calibration = pd.concat([predictions_lower_bound, predictions_upper_bound], axis=1)
prediction_interval_calibration.columns = ['lower_bound', 'upper_bound']
prediction_interval_calibration['y_true'] = data.loc[end_train:end_validation, 'OT']
prediction_interval_calibration.head(3)


  0%|          | 0/184 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

,lower_bound,upper_bound,y_true
2017-10-02 00:00:00,32.021783,32.883000,32.839251
2017-10-02 01:00:00,30.977375,32.623304,32.949501
2017-10-02 02:00:00,30.209657,32.436979,32.729626


Conformity score in Conformalized quantile regression:

$$s(y,x) == max(\hat{q}_{low}(X) - y_{true}, y_{true} - \hat{q}_{up}(X))$$

The score is positive if the true value y lies outside of the interval, and negative if it lies inside.

The threshold q can be interpreted as the term by which the interval has to be expanded (on both ends) or shortened. If 1-ɑ of the y’s are already within the interval (meaning the model is well calibrated), then the threshold would be at 0. A positive threshold means that the original intervals were too narrow, and a negative one that quantile intervals were too wide.

To compute the prediction interval, we add the threshold to the upper bound and subtract it from the lower bound (aka the old quantiles).

The calibration part of the procedure is not adaptive, since the same term will be added no matter the feature values. However, the entire procedure is adaptive since quantile regression is adaptive.

In [ ]:
# 2) Non-conformity score
# ==============================================================================
y_true = data.loc[end_train:end_validation, 'OT']
conformity_scores = np.max(
    [
        prediction_interval_calibration['lower_bound'] - y_true,
        y_true - prediction_interval_calibration['upper_bound'],
    ],
    axis=0,
)
conformity_scores

array([ -0.04374951,   0.32619728,   0.29264704, ..., -12.32276219,
       -13.83871647, -14.90388641])

In [ ]:
# 3) Correction factor
# ==============================================================================
emperical_quantile = 0.8
correction_factor = np.quantile(conformity_scores, emperical_quantile)
correction_factor

-0.4114107121275259

In [89]:
# 4) Backtesting on test data
# ==============================================================================
cv = TimeSeriesFold(
        initial_train_size = len(data.loc[:end_validation, :]),
        steps              = 24,  # all hours of next day
        differentiation    = 1
     )
_, predictions_lower_bound = backtesting_forecaster(
                                 forecaster          = forecaster_lower_bound,
                                 y                   = data['OT'],
                                 exog                = data[selected_exog],
                                 cv                  = cv,
                                 metric              = mean_pinball_loss_q10,
                                 n_jobs              = 'auto',
                                 verbose             = False,
                                 show_progress       = True
                              )

_, predictions_upper_bound = backtesting_forecaster(
                                  forecaster          = forecaster_upper_bound,
                                  y                   = data['OT'],
                                  exog                = data[selected_exog],
                                  cv                  = cv,
                                  metric              = mean_pinball_loss_q90,
                                  n_jobs              = 'auto',
                                  verbose             = False,
                                  show_progress       = True
                              )

prediction_interval_test = pd.concat([predictions_lower_bound, predictions_upper_bound], axis=1)
prediction_interval_test.columns = ['lower_bound', 'upper_bound']
prediction_interval_test['y_true'] = data.loc[end_validation:, 'OT']
prediction_interval_test.head(3)

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

,lower_bound,upper_bound,y_true
2018-04-04 00:00:00,32.342958,32.882199,32.674500
2018-04-04 01:00:00,31.496005,32.475559,31.575750
2018-04-04 02:00:00,30.717917,32.167618,29.763125


In [90]:
# 5) Conformal interval
# ==============================================================================
prediction_interval_test['lower_bound_conformal'] = prediction_interval_test['lower_bound'] - correction_factor
prediction_interval_test['upper_bound_conformal'] = prediction_interval_test['upper_bound'] + correction_factor
prediction_interval_test.head(3)

# TODO: esto me lo he inventado, no se si esta bien: If upper bound is less than lower bound, swap them
mask = prediction_interval_test['upper_bound_conformal'] < prediction_interval_test['lower_bound_conformal']
prediction_interval_test.loc[mask, 'upper_bound_conformal'], prediction_interval_test.loc[mask, 'lower_bound_conformal'] = (
    prediction_interval_test.loc[mask, 'lower_bound_conformal'],
    prediction_interval_test.loc[mask, 'upper_bound_conformal'],
)
prediction_interval_test.head(3)


,lower_bound,upper_bound,y_true,lower_bound_conformal,upper_bound_conformal
2018-04-04 00:00:00,32.342958,32.882199,32.674500,32.470788,32.754368
2018-04-04 01:00:00,31.496005,32.475559,31.575750,31.907415,32.064149
2018-04-04 02:00:00,30.717917,32.167618,29.763125,31.129328,31.756207


In [91]:
empirical_coverage = coverage(
                        y_true      = prediction_interval_test['y_true'].to_numpy(),
                        lower_bound = prediction_interval_test["lower_bound_conformal"].to_numpy(), 
                        upper_bound = prediction_interval_test["upper_bound_conformal"].to_numpy()
                    )
print(f"Predicted interval coverage: {round(100 * empirical_coverage, 2)} %")

# Area of the interval
# ==============================================================================
area = (prediction_interval_test["upper_bound_conformal"] - prediction_interval_test["lower_bound_conformal"]).sum()
print(f"Area of the interval: {round(area, 2)}")

Predicted interval coverage: 82.02 %
Area of the interval: 27444.05


In [ ]:
prediction_interval_test

,lower_bound,upper_bound,y_true,lower_bound_conformal,upper_bound_conformal
2018-04-04 00:00:00,32.342958,32.882199,32.674500,32.470788,32.754368
2018-04-04 01:00:00,31.496005,32.475559,31.575750,31.907415,32.064149
2018-04-04 02:00:00,30.717917,32.167618,29.763125,31.129328,31.756207
2018-04-04 03:00:00,29.984784,31.992088,27.895500,30.396195,31.580677
2018-04-04 04:00:00,29.341704,31.684803,26.302375,29.753115,31.273393
...,...,...,...,...,...
2018-06-26 16:00:00,39.368141,57.785450,47.744249,39.779552,57.374040
2018-06-26 17:00:00,37.243808,58.081516,48.183498,37.655219,57.670106
2018-06-26 18:00:00,35.973284,57.469137,47.853999,36.384695,57.057727
2018-06-26 19:00:00,34.637945,56.883772,46.535750,35.049356,56.472361


In [ ]:
# Plot
# ==============================================================================
fig = go.Figure([
    go.Scatter(name='Real value', x=data_test.index, y=data_test['OT'], mode='lines'),
    go.Scatter(
        name='Upper Bound', x=prediction_interval_test.index, y=prediction_interval_test['upper_bound_conformal'],
        mode='lines', marker=dict(color="rgba(243, 49, 0, 0.56)"), line=dict(width=0), showlegend=False
    ),
    go.Scatter(
        name='Lower Bound', x=prediction_interval_test.index, y=prediction_interval_test['lower_bound_conformal'],
        marker=dict(color="rgba(243, 49, 0, 0.56)"), line=dict(width=0), mode='lines',
        fillcolor='rgba(243, 49, 0, 0.56)', fill='tonexty', showlegend=False
    )
    ,
    go.Scatter(
        name='Upper Bound', x=prediction_interval_no_conformal.index, y=prediction_interval_no_conformal['upper_bound'],
        mode='lines', marker=dict(color="#444"), line=dict(width=0), showlegend=False
    ),
    go.Scatter(
        name='Lower Bound', x=prediction_interval_no_conformal.index, y=prediction_interval_no_conformal['lower_bound'],
        marker=dict(color="#444"), line=dict(width=0), mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)', fill='tonexty', showlegend=False
    )

])
fig.update_layout(
    title="Real value vs predicted in test data",
    xaxis_title="Date time",
    yaxis_title="OT",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    hovermode="x",
    legend=dict(orientation="h", yanchor="top", y=1.1, xanchor="left", x=0.001)
)
fig.show()

## ForecasterRecursive

In [99]:
forecaster = ForecasterRecursive(
                 regressor       = Ridge(random_state=15926, alpha=1.1),
                 lags            = lags,
                 differentiation = 1,
                 binner_kwargs   = {'n_bins': 10}
             )


In [100]:
# Backtesting on validation data to obtain out-sample residuals
# ==============================================================================
cv = TimeSeriesFold(
        initial_train_size = len(data.loc[:end_train, :]),
        steps              = 24,  # all hours of next day
        differentiation    = 1,
     )

metric_val, predictions_val = backtesting_forecaster(
                                forecaster    = forecaster,
                                y             = data.loc[:end_validation, 'OT'],
                                exog          = data.loc[:end_validation, selected_exog],
                                cv            = cv,
                                metric        = 'mean_absolute_error',
                                n_jobs        = 'auto',
                                verbose       = False,
                                show_progress = True
                              )

  0%|          | 0/184 [00:00<?, ?it/s]

In [101]:
# Store out-sample residuals in the forecaster
# ==============================================================================
forecaster.fit(y=data.loc[:end_train, 'OT'], exog=data.loc[:end_train, selected_exog])
forecaster.set_out_sample_residuals(
    y_true = data.loc[predictions_val.index, 'OT'], 
    y_pred = predictions_val['pred']
)

In [104]:
# Backtesting in test data
# ==============================================================================
cv = TimeSeriesFold(
        initial_train_size = len(data.loc[:end_validation, :]),
        steps              = 24,  # all hours of next day
        differentiation    = 1
     )

metric, predictions = backtesting_forecaster(
                          forecaster              = forecaster,
                          y                       = data['OT'],
                          exog                    = data[selected_exog],
                          cv                      = cv,
                          metric                  = 'mean_absolute_error',
                          n_jobs                  = 'auto',
                          verbose                 = False,
                          show_progress           = True
                     )
predictions

  0%|          | 0/84 [00:00<?, ?it/s]

,pred
2018-04-04 00:00:00,32.671269
2018-04-04 01:00:00,32.076115
2018-04-04 02:00:00,31.542983
2018-04-04 03:00:00,31.000276
2018-04-04 04:00:00,30.456563
...,...
2018-06-26 16:00:00,50.989807
2018-06-26 17:00:00,50.363266
2018-06-26 18:00:00,49.390910
2018-06-26 19:00:00,48.201490


In [109]:
# Create conformal interval
# ==============================================================================
correction_factor = np.quantile(np.abs(forecaster.out_sample_residuals_), 0.8)
correction_factor

0.9513869960775645

In [110]:
# Conformal interval
# ==============================================================================
prediction_interval = pd.DataFrame({
    'y_pred': predictions['pred'],
    'lower_bound': predictions['pred'] - correction_factor,
    'upper_bound': predictions['pred'] + correction_factor,
    'y_true': data.loc[end_validation:, 'OT']
})

prediction_interval

,y_pred,lower_bound,upper_bound,y_true
2018-04-04 00:00:00,32.671269,31.719882,33.622656,32.674500
2018-04-04 01:00:00,32.076115,31.124728,33.027502,31.575750
2018-04-04 02:00:00,31.542983,30.591596,32.494370,29.763125
2018-04-04 03:00:00,31.000276,30.048889,31.951663,27.895500
2018-04-04 04:00:00,30.456563,29.505176,31.407950,26.302375
...,...,...,...,...
2018-06-26 16:00:00,50.989807,50.038420,51.941194,47.744249
2018-06-26 17:00:00,50.363266,49.411879,51.314653,48.183498
2018-06-26 18:00:00,49.390910,48.439523,50.342297,47.853999
2018-06-26 19:00:00,48.201490,47.250103,49.152877,46.535750


In [111]:
# Predicted interval coverage (on test data)
# ==============================================================================
empirical_coverage = coverage(
                        y_true      = prediction_interval['y_true'].to_numpy(),
                        lower_bound = prediction_interval["lower_bound"].to_numpy(),
                        upper_bound = prediction_interval["upper_bound"].to_numpy()
                    )
print(f"Predicted interval coverage: {round(100 * empirical_coverage, 2)} %")

# Area of the interval
# ==============================================================================
area = (prediction_interval["upper_bound"] - prediction_interval["lower_bound"]).sum()
print(f"Area of the interval: {round(area, 2)}")

Predicted interval coverage: 35.17 %
Area of the interval: 3830.28
